# AdventureWorks - Predicting customer churn

In this notebook, we will demonstrate how getML can be used for a customer churn project using a synthetic dataset of a fictional company. We will also benchmark getML against featuretools.

Summary:

- Prediction type: __Classification model__
- Domain: __Customer loyalty__
- Prediction target: __churn__ 
- Population size: __19704__

_Author: Dr. Patrick Urbanke_

# Background

[AdventureWorks](https://github.com/Microsoft/sql-server-samples/releases/tag/adventureworks) is a fictional company that sells bicycles. It is used by Microsoft to showcase how its MS SQL Server can be used to manage business data. Since this dataset resembles a real-world customer database and it is open-source, we will use it to showcase how getML can be used for a classic customer churn project (real customer databases are not easily available for the purposes of showcasing and benchmarking, for reasons of data privacy).

The dataset has been downloaded from the [CTU Prague relational learning repository](https://relational.fit.cvut.cz/dataset/AdventureWorks) (Motl and Schulte, 2015).

We will benchmark [getML](https://www.getml.com) 's feature learning algorithms against [featuretools](https://www.featuretools.com), an open-source implementation of the propositionalization algorithm, similar to getML's FastProp.

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

Let's get started with the analysis and set up your session:

In [39]:
import copy
import datetime
import os
from urllib import request
import time

import numpy as np
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline  

import pyspark
import getml

getml.set_project('adventure_works')



Connected to project 'adventure_works'


## 1. Loading data

### 1.1 Download from source

We begin by downloading the data:

In [2]:
conn = getml.database.connect_mariadb(
    host="relational.fit.cvut.cz",
    dbname="AdventureWorks2014",
    port=3306,
    user="guest",
    password="relational"
)

conn

Connection(conn_id='default',
           dbname='AdventureWorks2014',
           dialect='mysql',
           host='relational.fit.cvut.cz',
           port=3306)

In [3]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if not getml.data.exists(name):
        data_frame = getml.DataFrame.from_db(
            name=name,
            table_name=name,
            conn=conn
        )
        data_frame.save()
    else:
        data_frame = getml.data.load_data_frame(name)
    return data_frame

In [4]:
product = load_if_needed("Product")
sales_order_detail = load_if_needed("SalesOrderDetail")
sales_order_header = load_if_needed("SalesOrderHeader")
sales_order_reason = load_if_needed("SalesOrderHeaderSalesReason")
special_offer = load_if_needed("SpecialOffer")
store = load_if_needed("Store")

In [5]:
product

name,ProductID,MakeFlag,ProductSubcategoryID,ProductModelID,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,FinishedGoodsFlag,DaysToManufacture,Name,ProductNumber,Color,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,ProductLine,Class,Style,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
role,join_key,categorical,categorical,categorical,numerical,numerical,numerical,numerical,unused_float,unused_float,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
0,1,0,NULL,NULL,1000,750,0,0,0,0,Adjustable Race,AR-5381,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,694215B7-08F7-4C0D-ACB1-D734BA44...,2014-02-08 10:01:36
1,2,0,NULL,NULL,1000,750,0,0,0,0,Bearing Ball,BA-8327,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,58AE3C20-4F3A-4749-A7D4-D568806C...,2014-02-08 10:01:36
2,3,1,NULL,NULL,800,600,0,0,0,1,BB Ball Bearing,BE-2349,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,9C21AED2-5BFA-4F18-BCB8-F11638DC...,2014-02-08 10:01:36
3,4,0,NULL,NULL,800,600,0,0,0,0,Headset Ball Bearings,BE-2908,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,ECFED6CB-51FF-49B5-B06C-7D8AC834...,2014-02-08 10:01:36
4,316,1,NULL,NULL,800,600,0,0,0,1,Blade,BL-2036,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,E73E9750-603B-4131-89F5-3DD15ED5...,2014-02-08 10:01:36
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,1,5,96,500,375,44.9506,101.24,1,1,ML Bottom Bracket,BB-8107,NULL,NULL,NULL,G,168.00,NULL,M,NULL,2013-05-30 00:00:00,NULL,NULL,71AB847F-D091-42D6-B735-7B0C2D82...,2014-02-08 10:01:36
500,996,1,5,97,500,375,53.9416,121.49,1,1,HL Bottom Bracket,BB-9108,NULL,NULL,NULL,G,170.00,NULL,H,NULL,2013-05-30 00:00:00,NULL,NULL,230C47C5-08B2-4CE3-B706-69C0BDD6...,2014-02-08 10:01:36
501,997,1,2,31,100,75,343.6496,539.99,1,4,"Road-750 Black, 44",BK-R19B-44,Black,44,CM,LB,19.77,R,L,U,2013-05-30 00:00:00,NULL,NULL,44CE4802-409F-43AB-9B27-CA534218...,2014-02-08 10:01:36


In [6]:
sales_order_detail

name,ModifiedDate,SalesOrderID,SalesOrderDetailID,ProductID,SpecialOfferID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,CarrierTrackingNumber,rowguid
role,time_stamp,join_key,join_key,join_key,join_key,numerical,numerical,numerical,numerical,unused_string,unused_string
unit,"time stamp, comparison only",,,,,,,,,,
0,2011-05-31,43659,1,776,1,1,2024.994,0,2024.994,4911-403C-98,B207C96D-D9E6-402B-8470-2CC176C4...
1,2011-05-31,43659,2,777,1,3,2024.994,0,6074.982,4911-403C-98,7ABB600D-1E77-41BE-9FE5-B9142CFC...
2,2011-05-31,43659,3,778,1,1,2024.994,0,2024.994,4911-403C-98,475CF8C6-49F6-486E-B0AD-AFC6A50C...
3,2011-05-31,43659,4,771,1,1,2039.994,0,2039.994,4911-403C-98,04C4DE91-5815-45D6-8670-F462719F...
4,2011-05-31,43659,5,772,1,1,2039.994,0,2039.994,4911-403C-98,5A74C7D2-E641-438E-A7AC-37BF2328...
,...,...,...,...,...,...,...,...,...,...,...
121312,2014-06-30,75122,121313,878,1,1,21.98,0,21.98,NULL,8CAD6675-18CC-4F47-8287-97B41A8E...
121313,2014-06-30,75122,121314,712,1,1,8.99,0,8.99,NULL,84F1C363-1C50-4442-BE16-541C59B6...


In [7]:
sales_order_header

name,OrderDate,DueDate,ShipDate,ModifiedDate,CustomerID,SalesOrderID,SalesPersonID,TerritoryID,RevisionNumber,OnlineOrderFlag,SalesPersonIDCat,TerritoryIDCat,ShipMethodID,SubTotal,TaxAmt,Freight,TotalDue,Status,BillToAddressID,ShipToAddressID,CreditCardID,CurrencyRateID,churn,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CreditCardApprovalCode,Comment,rowguid
role,time_stamp,time_stamp,time_stamp,time_stamp,join_key,join_key,join_key,join_key,categorical,categorical,categorical,categorical,categorical,numerical,numerical,numerical,numerical,unused_float,unused_float,unused_float,unused_float,unused_float,unused_float,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
unit,"time stamp, comparison only","time stamp, comparison only","time stamp, comparison only","time stamp, comparison only",,,,,,,,,,,,,,,,,,,,,,,,,
0,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29825,43659,279,5,8,0,279,5,5,20565.6206,1971.5149,616.0984,23153.2339,5,985,985,16281,nan,0,SO43659,PO522145787,10-4020-000676,105041Vi84182,NULL,79B65321-39CA-4115-9CBA-8FE0903E...
1,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29672,43660,279,5,8,0,279,5,5,1294.2529,124.2483,38.8276,1457.3288,5,921,921,5618,nan,0,SO43660,PO18850127500,10-4020-000117,115213Vi29411,NULL,738DC42D-D03B-48A1-9822-F95A67EA...
2,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29734,43661,282,6,8,0,282,6,5,32726.4786,3153.7696,985.553,36865.8012,5,517,517,1346,4,0,SO43661,PO18473189620,10-4020-000442,85274Vi6854,NULL,D91B9131-18A4-4A11-BC3A-90B6F53E...
3,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29994,43662,282,6,8,0,282,6,5,28832.5289,2775.1646,867.2389,32474.9324,5,482,482,10456,4,0,SO43662,PO18444174044,10-4020-000227,125295Vi53935,NULL,4A1ECFC0-CC3A-4740-B028-1C50BB48...
4,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29565,43663,276,4,8,0,276,4,5,419.4589,40.2681,12.5838,472.3108,5,1073,1073,4322,nan,0,SO43663,PO18009186470,10-4020-000510,45303Vi22691,NULL,9B1E7A40-6AE0-4AD3-811C-A6495185...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31460,2014-06-30,2014-07-12,2014-07-07,2014-07-07,11981,75119,NULL,1,8,1,NULL,1,1,42.28,3.3824,1.057,46.7194,5,17649,17649,6761,nan,nan,SO75119,NULL,10-4030-011981,429826Vi35166,NULL,9382F1C9-383A-435F-9449-0EECEA21...
31461,2014-06-30,2014-07-12,2014-07-07,2014-07-07,18749,75120,NULL,6,8,1,NULL,6,1,84.96,6.7968,2.124,93.8808,5,28374,28374,8925,nan,nan,SO75120,NULL,10-4030-018749,929849Vi46003,NULL,AE6A4FCF-FF73-4CD4-AF2C-5993D00D...


In [8]:
sales_order_reason

name,SalesOrderID,SalesReasonID,ModifiedDate
role,join_key,categorical,unused_string
0,43697,5,2011-05-31 00:00:00
1,43697,9,2011-05-31 00:00:00
2,43702,5,2011-06-01 00:00:00
3,43702,9,2011-06-01 00:00:00
4,43703,5,2011-06-01 00:00:00
,...,...,...
27642,75119,1,2014-06-30 00:00:00
27643,75120,1,2014-06-30 00:00:00
27644,75121,1,2014-06-30 00:00:00


In [9]:
special_offer

name,StartDate,EndDate,SpecialOfferID,Category,Description,Type,MinQty,DiscountPct,MaxQty,rowguid,ModifiedDate
role,time_stamp,time_stamp,join_key,categorical,categorical,categorical,numerical,numerical,unused_float,unused_string,unused_string
unit,"time stamp, comparison only","time stamp, comparison only",,,,,,,,,
0,2011-05-01,2014-11-30,1,No Discount,No Discount,No Discount,0,0,nan,0290C4F5-191F-4337-AB6B-0A2DDE03...,2011-04-01 00:00:00
1,2011-05-31,2014-05-30,2,Reseller,Volume Discount 11 to 14,Volume Discount,11,0.02,14,D7542EE7-15DB-4541-985C-5CC27AEF...,2011-05-01 00:00:00
2,2011-05-31,2014-05-30,3,Reseller,Volume Discount 15 to 24,Volume Discount,15,0.05,24,4BDBCC01-8CF7-40A9-B643-40EC5B71...,2011-05-01 00:00:00
3,2011-05-31,2014-05-30,4,Reseller,Volume Discount 25 to 40,Volume Discount,25,0.1,40,504B5E85-8F3F-4EBC-9E1D-C1BC5DEA...,2011-05-01 00:00:00
4,2011-05-31,2014-05-30,5,Reseller,Volume Discount 41 to 60,Volume Discount,41,0.15,60,677E1D9D-944F-4E81-90E8-47EB0A82...,2011-05-01 00:00:00
,...,...,...,...,...,...,...,...,...,...,...
11,2013-05-30,2013-07-14,12,Reseller,LL Road Frame Sale,Excess Inventory,0,0.35,nan,C0AF1C89-9722-4235-9248-3FBA4D9E...,2013-04-30 00:00:00
12,2013-05-30,2013-08-29,13,Reseller,Touring-3000 Promotion,New Product,0,0.15,nan,5061CCE4-E021-45A8-9A75-DFB36CBB...,2013-04-30 00:00:00


In [10]:
store

name,SalesPersonID,BusinessEntityID,Name,Demographics,rowguid,ModifiedDate
role,join_key,unused_float,unused_string,unused_string,unused_string,unused_string
0,279,292,Next-Door Bike Store,"<StoreSurvey xmlns=""http://schem...",A22517E3-848D-4EBE-B9D9-7437F343...,2014-09-12 11:15:07
1,276,294,Professional Sales and Service,"<StoreSurvey xmlns=""http://schem...",B50CA50B-C601-4A13-B07E-2C63862D...,2014-09-12 11:15:07
2,277,296,Riders Company,"<StoreSurvey xmlns=""http://schem...",337C3688-1339-4E1A-A08A-B54B2356...,2014-09-12 11:15:07
3,275,298,The Bike Mechanics,"<StoreSurvey xmlns=""http://schem...",7894F278-F0C8-4D16-BD75-213FDBF1...,2014-09-12 11:15:07
4,286,300,Nationwide Supply,"<StoreSurvey xmlns=""http://schem...",C3FC9705-A8C4-4F3A-9550-EB2FA4B7...,2014-09-12 11:15:07
,...,...,...,...,...,...
696,282,1988,Retreat Inn,"<StoreSurvey xmlns=""http://schem...",EA21EC81-1BFA-4A07-9B4D-73D9852A...,2014-09-12 11:15:07
697,281,1990,Technical Parts Manufacturing,"<StoreSurvey xmlns=""http://schem...",C8E3C4ED-8F58-4DB2-B600-E0CD11D9...,2014-09-12 11:15:07
698,277,1992,Totes & Baskets Company,"<StoreSurvey xmlns=""http://schem...",CE860B58-643C-4567-BFD8-06E97969...,2014-09-12 11:15:07


### 1.2 Prepare data for getML

getML requires that we define *roles* for each of the columns.

In [11]:
product.set_role("ProductID", getml.data.roles.join_key)
product.set_role(["MakeFlag", "ProductSubcategoryID", "ProductModelID"], getml.data.roles.categorical)
product.set_role(["SafetyStockLevel", "ReorderPoint", "StandardCost", "ListPrice"], getml.data.roles.numerical)

product

name,ProductID,MakeFlag,ProductSubcategoryID,ProductModelID,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,FinishedGoodsFlag,DaysToManufacture,Name,ProductNumber,Color,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,ProductLine,Class,Style,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
role,join_key,categorical,categorical,categorical,numerical,numerical,numerical,numerical,unused_float,unused_float,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
0,1,0,NULL,NULL,1000,750,0,0,0,0,Adjustable Race,AR-5381,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,694215B7-08F7-4C0D-ACB1-D734BA44...,2014-02-08 10:01:36
1,2,0,NULL,NULL,1000,750,0,0,0,0,Bearing Ball,BA-8327,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,58AE3C20-4F3A-4749-A7D4-D568806C...,2014-02-08 10:01:36
2,3,1,NULL,NULL,800,600,0,0,0,1,BB Ball Bearing,BE-2349,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,9C21AED2-5BFA-4F18-BCB8-F11638DC...,2014-02-08 10:01:36
3,4,0,NULL,NULL,800,600,0,0,0,0,Headset Ball Bearings,BE-2908,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,ECFED6CB-51FF-49B5-B06C-7D8AC834...,2014-02-08 10:01:36
4,316,1,NULL,NULL,800,600,0,0,0,1,Blade,BL-2036,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2008-04-30 00:00:00,NULL,NULL,E73E9750-603B-4131-89F5-3DD15ED5...,2014-02-08 10:01:36
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,1,5,96,500,375,44.9506,101.24,1,1,ML Bottom Bracket,BB-8107,NULL,NULL,NULL,G,168.00,NULL,M,NULL,2013-05-30 00:00:00,NULL,NULL,71AB847F-D091-42D6-B735-7B0C2D82...,2014-02-08 10:01:36
500,996,1,5,97,500,375,53.9416,121.49,1,1,HL Bottom Bracket,BB-9108,NULL,NULL,NULL,G,170.00,NULL,H,NULL,2013-05-30 00:00:00,NULL,NULL,230C47C5-08B2-4CE3-B706-69C0BDD6...,2014-02-08 10:01:36
501,997,1,2,31,100,75,343.6496,539.99,1,4,"Road-750 Black, 44",BK-R19B-44,Black,44,CM,LB,19.77,R,L,U,2013-05-30 00:00:00,NULL,NULL,44CE4802-409F-43AB-9B27-CA534218...,2014-02-08 10:01:36


In [12]:
sales_order_detail.set_role(["SalesOrderID", "SalesOrderDetailID", "ProductID", "SpecialOfferID"], getml.data.roles.join_key)
sales_order_detail.set_role(["OrderQty", "UnitPrice", "UnitPriceDiscount", "LineTotal"], getml.data.roles.numerical)
sales_order_detail.set_role("ModifiedDate", getml.data.roles.time_stamp)

sales_order_detail

name,ModifiedDate,SalesOrderID,SalesOrderDetailID,ProductID,SpecialOfferID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,CarrierTrackingNumber,rowguid
role,time_stamp,join_key,join_key,join_key,join_key,numerical,numerical,numerical,numerical,unused_string,unused_string
unit,"time stamp, comparison only",,,,,,,,,,
0,2011-05-31,43659,1,776,1,1,2024.994,0,2024.994,4911-403C-98,B207C96D-D9E6-402B-8470-2CC176C4...
1,2011-05-31,43659,2,777,1,3,2024.994,0,6074.982,4911-403C-98,7ABB600D-1E77-41BE-9FE5-B9142CFC...
2,2011-05-31,43659,3,778,1,1,2024.994,0,2024.994,4911-403C-98,475CF8C6-49F6-486E-B0AD-AFC6A50C...
3,2011-05-31,43659,4,771,1,1,2039.994,0,2039.994,4911-403C-98,04C4DE91-5815-45D6-8670-F462719F...
4,2011-05-31,43659,5,772,1,1,2039.994,0,2039.994,4911-403C-98,5A74C7D2-E641-438E-A7AC-37BF2328...
,...,...,...,...,...,...,...,...,...,...,...
121312,2014-06-30,75122,121313,878,1,1,21.98,0,21.98,NULL,8CAD6675-18CC-4F47-8287-97B41A8E...
121313,2014-06-30,75122,121314,712,1,1,8.99,0,8.99,NULL,84F1C363-1C50-4442-BE16-541C59B6...


In [13]:
sales_order_reason.set_role("SalesOrderID", getml.data.roles.join_key)
sales_order_reason.set_role("SalesReasonID", getml.data.roles.categorical)

sales_order_reason

name,SalesOrderID,SalesReasonID,ModifiedDate
role,join_key,categorical,unused_string
0,43697,5,2011-05-31 00:00:00
1,43697,9,2011-05-31 00:00:00
2,43702,5,2011-06-01 00:00:00
3,43702,9,2011-06-01 00:00:00
4,43703,5,2011-06-01 00:00:00
,...,...,...
27642,75119,1,2014-06-30 00:00:00
27643,75120,1,2014-06-30 00:00:00
27644,75121,1,2014-06-30 00:00:00


In [14]:
special_offer.set_role(["SpecialOfferID"], getml.data.roles.join_key)
special_offer.set_role(["MinQty", "DiscountPct"], getml.data.roles.numerical)
special_offer.set_role(["Category", "Description", "Type"], getml.data.roles.categorical)
special_offer.set_role(["StartDate", "EndDate"], getml.data.roles.time_stamp)

special_offer

name,StartDate,EndDate,SpecialOfferID,Category,Description,Type,MinQty,DiscountPct,MaxQty,rowguid,ModifiedDate
role,time_stamp,time_stamp,join_key,categorical,categorical,categorical,numerical,numerical,unused_float,unused_string,unused_string
unit,"time stamp, comparison only","time stamp, comparison only",,,,,,,,,
0,2011-05-01,2014-11-30,1,No Discount,No Discount,No Discount,0,0,nan,0290C4F5-191F-4337-AB6B-0A2DDE03...,2011-04-01 00:00:00
1,2011-05-31,2014-05-30,2,Reseller,Volume Discount 11 to 14,Volume Discount,11,0.02,14,D7542EE7-15DB-4541-985C-5CC27AEF...,2011-05-01 00:00:00
2,2011-05-31,2014-05-30,3,Reseller,Volume Discount 15 to 24,Volume Discount,15,0.05,24,4BDBCC01-8CF7-40A9-B643-40EC5B71...,2011-05-01 00:00:00
3,2011-05-31,2014-05-30,4,Reseller,Volume Discount 25 to 40,Volume Discount,25,0.1,40,504B5E85-8F3F-4EBC-9E1D-C1BC5DEA...,2011-05-01 00:00:00
4,2011-05-31,2014-05-30,5,Reseller,Volume Discount 41 to 60,Volume Discount,41,0.15,60,677E1D9D-944F-4E81-90E8-47EB0A82...,2011-05-01 00:00:00
,...,...,...,...,...,...,...,...,...,...,...
11,2013-05-30,2013-07-14,12,Reseller,LL Road Frame Sale,Excess Inventory,0,0.35,nan,C0AF1C89-9722-4235-9248-3FBA4D9E...,2013-04-30 00:00:00
12,2013-05-30,2013-08-29,13,Reseller,Touring-3000 Promotion,New Product,0,0.15,nan,5061CCE4-E021-45A8-9A75-DFB36CBB...,2013-04-30 00:00:00


In [15]:
store.set_role(["SalesPersonID"], getml.data.roles.join_key)
store.set_role(["SalesPersonID"], getml.data.roles.join_key)
store

name,SalesPersonID,BusinessEntityID,Name,Demographics,rowguid,ModifiedDate
role,join_key,unused_float,unused_string,unused_string,unused_string,unused_string
0,279,292,Next-Door Bike Store,"<StoreSurvey xmlns=""http://schem...",A22517E3-848D-4EBE-B9D9-7437F343...,2014-09-12 11:15:07
1,276,294,Professional Sales and Service,"<StoreSurvey xmlns=""http://schem...",B50CA50B-C601-4A13-B07E-2C63862D...,2014-09-12 11:15:07
2,277,296,Riders Company,"<StoreSurvey xmlns=""http://schem...",337C3688-1339-4E1A-A08A-B54B2356...,2014-09-12 11:15:07
3,275,298,The Bike Mechanics,"<StoreSurvey xmlns=""http://schem...",7894F278-F0C8-4D16-BD75-213FDBF1...,2014-09-12 11:15:07
4,286,300,Nationwide Supply,"<StoreSurvey xmlns=""http://schem...",C3FC9705-A8C4-4F3A-9550-EB2FA4B7...,2014-09-12 11:15:07
,...,...,...,...,...,...
696,282,1988,Retreat Inn,"<StoreSurvey xmlns=""http://schem...",EA21EC81-1BFA-4A07-9B4D-73D9852A...,2014-09-12 11:15:07
697,281,1990,Technical Parts Manufacturing,"<StoreSurvey xmlns=""http://schem...",C8E3C4ED-8F58-4DB2-B600-E0CD11D9...,2014-09-12 11:15:07
698,277,1992,Totes & Baskets Company,"<StoreSurvey xmlns=""http://schem...",CE860B58-643C-4567-BFD8-06E97969...,2014-09-12 11:15:07


In [16]:
sales_order_header["SalesPersonIDCat"] = sales_order_header["SalesPersonID"]
sales_order_header["TerritoryIDCat"] = sales_order_header["TerritoryID"]

sales_order_header.set_role(["CustomerID", "SalesOrderID", "SalesPersonID", "TerritoryID"], getml.data.roles.join_key)
sales_order_header.set_role(
    ["RevisionNumber", "OnlineOrderFlag", "SalesPersonIDCat", "TerritoryIDCat", "ShipMethodID"], 
    getml.data.roles.categorical)
sales_order_header.set_role(["SubTotal", "TaxAmt", "Freight", "TotalDue"], getml.data.roles.numerical)
sales_order_header.set_role(["OrderDate", "DueDate", "ShipDate", "ModifiedDate"], getml.data.roles.time_stamp)

sales_order_header

name,OrderDate,DueDate,ShipDate,ModifiedDate,CustomerID,SalesOrderID,SalesPersonID,TerritoryID,RevisionNumber,OnlineOrderFlag,SalesPersonIDCat,TerritoryIDCat,ShipMethodID,SubTotal,TaxAmt,Freight,TotalDue,Status,BillToAddressID,ShipToAddressID,CreditCardID,CurrencyRateID,churn,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CreditCardApprovalCode,Comment,rowguid
role,time_stamp,time_stamp,time_stamp,time_stamp,join_key,join_key,join_key,join_key,categorical,categorical,categorical,categorical,categorical,numerical,numerical,numerical,numerical,unused_float,unused_float,unused_float,unused_float,unused_float,unused_float,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
unit,"time stamp, comparison only","time stamp, comparison only","time stamp, comparison only","time stamp, comparison only",,,,,,,,,,,,,,,,,,,,,,,,,
0,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29825,43659,279,5,8,0,279,5,5,20565.6206,1971.5149,616.0984,23153.2339,5,985,985,16281,nan,0,SO43659,PO522145787,10-4020-000676,105041Vi84182,NULL,79B65321-39CA-4115-9CBA-8FE0903E...
1,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29672,43660,279,5,8,0,279,5,5,1294.2529,124.2483,38.8276,1457.3288,5,921,921,5618,nan,0,SO43660,PO18850127500,10-4020-000117,115213Vi29411,NULL,738DC42D-D03B-48A1-9822-F95A67EA...
2,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29734,43661,282,6,8,0,282,6,5,32726.4786,3153.7696,985.553,36865.8012,5,517,517,1346,4,0,SO43661,PO18473189620,10-4020-000442,85274Vi6854,NULL,D91B9131-18A4-4A11-BC3A-90B6F53E...
3,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29994,43662,282,6,8,0,282,6,5,28832.5289,2775.1646,867.2389,32474.9324,5,482,482,10456,4,0,SO43662,PO18444174044,10-4020-000227,125295Vi53935,NULL,4A1ECFC0-CC3A-4740-B028-1C50BB48...
4,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29565,43663,276,4,8,0,276,4,5,419.4589,40.2681,12.5838,472.3108,5,1073,1073,4322,nan,0,SO43663,PO18009186470,10-4020-000510,45303Vi22691,NULL,9B1E7A40-6AE0-4AD3-811C-A6495185...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31460,2014-06-30,2014-07-12,2014-07-07,2014-07-07,11981,75119,NULL,1,8,1,NULL,1,1,42.28,3.3824,1.057,46.7194,5,17649,17649,6761,nan,nan,SO75119,NULL,10-4030-011981,429826Vi35166,NULL,9382F1C9-383A-435F-9449-0EECEA21...
31461,2014-06-30,2014-07-12,2014-07-07,2014-07-07,18749,75120,NULL,6,8,1,NULL,6,1,84.96,6.7968,2.124,93.8808,5,28374,28374,8925,nan,nan,SO75120,NULL,10-4030-018749,929849Vi46003,NULL,AE6A4FCF-FF73-4CD4-AF2C-5993D00D...


We must also define customer churn. In this case, we define customer churn as a customer not making another purchase within 180 days of his or her last purchase.

Thus, the churn variable is defined as follows:

- 0, if another purchase by the same customer has been made within 180 days after `OrderDate`
- 1, if no purchase by the same customer has been made within 180 days after `OrderDate`
- NULL, if `max(OrderDate) - OrderDate <= 180 days`

NULL targets can not be used in our analysis.

In [17]:
sales_order_header_pd = sales_order_header[["OrderDate", "CustomerID", "SalesOrderID"]].to_pandas()

repeat_purchases = sales_order_header_pd.merge(
    sales_order_header_pd[["OrderDate", "CustomerID"]],
    on="CustomerID",
    how="left",
)

repeat_purchases = repeat_purchases[
    repeat_purchases["OrderDate_y"] > repeat_purchases["OrderDate_x"]
]

repeat_purchases = repeat_purchases[
    repeat_purchases["OrderDate_y"] - repeat_purchases["OrderDate_x"] > pd.Timedelta('180 days')
]

repeat_purchases.groupby("SalesOrderID", as_index=False).aggregate({"CustomerID": "max"})

repeat_purchase_ids = {sid: True for sid in repeat_purchases["SalesOrderID"]}

cut_off_date = max(sales_order_header_pd["OrderDate"]) - pd.Timedelta('180 days')

churn = np.asarray([
    np.nan if order_date >= cut_off_date else 0 if order_id in repeat_purchase_ids else 1 
    for (order_date, order_id) in zip(sales_order_header_pd["OrderDate"], sales_order_header_pd["SalesOrderID"])
])

sales_order_header["churn"] = churn

sales_order_header = sales_order_header[~sales_order_header.churn.is_nan()].to_df("SalesOrderHeaderRefined")

sales_order_header.set_role("churn", getml.data.roles.target)

sales_order_header

name,OrderDate,DueDate,ShipDate,ModifiedDate,CustomerID,SalesOrderID,SalesPersonID,TerritoryID,churn,RevisionNumber,OnlineOrderFlag,SalesPersonIDCat,TerritoryIDCat,ShipMethodID,SubTotal,TaxAmt,Freight,TotalDue,Status,BillToAddressID,ShipToAddressID,CreditCardID,CurrencyRateID,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CreditCardApprovalCode,Comment,rowguid
role,time_stamp,time_stamp,time_stamp,time_stamp,join_key,join_key,join_key,join_key,target,categorical,categorical,categorical,categorical,categorical,numerical,numerical,numerical,numerical,unused_float,unused_float,unused_float,unused_float,unused_float,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
unit,"time stamp, comparison only","time stamp, comparison only","time stamp, comparison only","time stamp, comparison only",,,,,,,,,,,,,,,,,,,,,,,,,
0,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29825,43659,279,5,0,8,0,279,5,5,20565.6206,1971.5149,616.0984,23153.2339,5,985,985,16281,nan,SO43659,PO522145787,10-4020-000676,105041Vi84182,NULL,79B65321-39CA-4115-9CBA-8FE0903E...
1,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29672,43660,279,5,0,8,0,279,5,5,1294.2529,124.2483,38.8276,1457.3288,5,921,921,5618,nan,SO43660,PO18850127500,10-4020-000117,115213Vi29411,NULL,738DC42D-D03B-48A1-9822-F95A67EA...
2,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29734,43661,282,6,0,8,0,282,6,5,32726.4786,3153.7696,985.553,36865.8012,5,517,517,1346,4,SO43661,PO18473189620,10-4020-000442,85274Vi6854,NULL,D91B9131-18A4-4A11-BC3A-90B6F53E...
3,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29994,43662,282,6,0,8,0,282,6,5,28832.5289,2775.1646,867.2389,32474.9324,5,482,482,10456,4,SO43662,PO18444174044,10-4020-000227,125295Vi53935,NULL,4A1ECFC0-CC3A-4740-B028-1C50BB48...
4,2011-05-31,2011-06-12,2011-06-07,2011-06-07,29565,43663,276,4,0,8,0,276,4,5,419.4589,40.2681,12.5838,472.3108,5,1073,1073,4322,nan,SO43663,PO18009186470,10-4020-000510,45303Vi22691,NULL,9B1E7A40-6AE0-4AD3-811C-A6495185...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19699,2013-12-31,2014-01-12,2014-01-07,2014-01-07,20826,63358,NULL,7,1,8,1,NULL,7,1,1173.96,93.9168,29.349,1297.2258,5,24387,24387,3239,nan,SO63358,NULL,10-4030-020826,1142084Vi17039,NULL,41278FBB-3DD8-488B-AEA5-8BF4A6F1...
19700,2013-12-31,2014-01-12,2014-01-07,2014-01-07,24114,63359,NULL,10,1,8,1,NULL,10,1,1179.47,94.3576,29.4868,1303.3144,5,29682,29682,nan,10770,SO63359,NULL,10-4030-024114,NULL,NULL,749532D4-BFF7-4FEC-9F77-396C4A96...


In [18]:
split = getml.data.split.random(train=0.8, test=0.2)
split

,
0,train
1,train
2,train
3,test
4,train
,...


In [19]:
container = getml.data.Container(population=sales_order_header, split=split)

container.add(
    product=product,
    sales_order_detail=sales_order_detail, 
    sales_order_header=sales_order_header, 
    sales_order_reason=sales_order_reason,
    special_offer=special_offer,
    store=store,
)

container

,subset,name,rows,type
0,test,SalesOrderHeaderRefined,3879,View
1,train,SalesOrderHeaderRefined,15825,View
,alias,name,rows,type
0,product,Product,504,DataFrame
1,sales_order_detail,SalesOrderDetail,121317,DataFrame
2,sales_order_header,SalesOrderHeaderRefined,19704,DataFrame
3,sales_order_reason,SalesOrderHeaderSalesReason,27647,DataFrame
4,special_offer,SpecialOffer,16,DataFrame
5,store,Store,701,DataFrame


## 2. Predictive modeling

We loaded the data and defined the roles and units. Next, we create a getML pipeline for relational learning.

### 2.1 Define relational model

In [20]:
dm = getml.data.DataModel(sales_order_header.to_placeholder("population"))

dm.add(getml.data.to_placeholder(
    product=product,
    sales_order_detail=sales_order_detail, 
    sales_order_header=sales_order_header, 
    sales_order_reason=sales_order_reason,
    special_offer=special_offer,
    store=store,
))


dm.population.join(
    dm.sales_order_header,
    on="CustomerID",
    time_stamps="OrderDate",
    lagged_targets=True,
    horizon=getml.data.time.days(1),
)

dm.population.join(
    dm.sales_order_detail,
    on="SalesOrderID",
)

dm.population.join(
    dm.sales_order_reason,
    on="SalesOrderID", 
)

dm.population.join(
    dm.store,
    on="SalesPersonID", 
)

dm.sales_order_detail.join(
    dm.product,
    on="ProductID",
    relationship=getml.data.relationship.many_to_one,
)

dm.sales_order_detail.join(
    dm.special_offer,
    on="SpecialOfferID",
    relationship=getml.data.relationship.many_to_one,
)

dm

,data frames,staging table
0,population,POPULATION__STAGING_TABLE_1
1,"sales_order_detail, product, special_offer",SALES_ORDER_DETAIL__STAGING_TABLE_2
2,sales_order_header,SALES_ORDER_HEADER__STAGING_TABLE_3
3,sales_order_reason,SALES_ORDER_REASON__STAGING_TABLE_4
4,store,STORE__STAGING_TABLE_5


### 2.2 getML pipeline

<!-- #### 2.1.1  -->
__Set-up the feature learner & predictor__

In [21]:
seasonal = getml.preprocessors.Seasonal()

mapping = getml.preprocessors.Mapping()

fast_prop = getml.feature_learning.FastProp(
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    num_threads=1,    
    num_features=400,
)

relboost = getml.feature_learning.Relboost(
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    num_threads=1,    
)

predictor = getml.predictors.XGBoostClassifier(n_jobs=1)

__Build the pipeline__

In [22]:
pipe1 = getml.Pipeline(
    tags=['fast_prop'],
    data_model=dm,
    preprocessors=[mapping],
    feature_learners=[fast_prop],
    predictors=[predictor],
    include_categorical=True,
)

pipe1

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=['product', 'sales_order_detail', 'sales_order_header',
                     'sales_order_reason', 'special_offer', 'store'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop'])

In [23]:
pipe2 = getml.Pipeline(
    tags=['relboost'],
    data_model=dm,
    preprocessors=[seasonal, mapping],
    feature_learners=[relboost],
    predictors=[predictor],
    include_categorical=True,
)

pipe2

Pipeline(data_model='population',
         feature_learners=['Relboost'],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=['product', 'sales_order_detail', 'sales_order_header',
                     'sales_order_reason', 'special_offer', 'store'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Seasonal', 'Mapping'],
         share_selected_features=0.5,
         tags=['relboost'])

### 2.3 Model training

In [24]:
pipe1.check(container.train)

Checking data model...


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

Checking...
[========================================] 100%

INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and SALES_ORDER_REASON__STAGING_TABLE_4 over 'SalesOrderID' and 'SalesOrderID', there are no corresponding entries for 33.769352% of entries in 'SalesOrderID' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and STORE__STAGING_TABLE_5 over 'SalesPersonID' and 'SalesPersonID', there are no corresponding entries for 84.941548% of entries in 'SalesPersonID' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.


In [25]:
pipe1.fit(container.train)

Checking data model...


Staging...
[========================================] 100%

INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and SALES_ORDER_REASON__STAGING_TABLE_4 over 'SalesOrderID' and 'SalesOrderID', there are no corresponding entries for 33.769352% of entries in 'SalesOrderID' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and STORE__STAGING_TABLE_5 over 'SalesPersonID' and 'SalesPersonID', there are no corresponding entries for 84.941548% of entries in 'SalesPersonID' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

FastProp: Trying 2076 features...
[========================================] 100%

FastProp: Building features...
[========================================] 100%

XGBoost: T

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=['product', 'sales_order_detail', 'sales_order_header',
                     'sales_order_reason', 'special_offer', 'store'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop', 'container-EORt4D'])

url: http://localhost:1709/#/getpipeline/adventure_works/NbfWvX/0/

In [26]:
pipe2.check(container.train)

Checking data model...


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

Checking...
[========================================] 100%

INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and SALES_ORDER_REASON__STAGING_TABLE_4 over 'SalesOrderID' and 'SalesOrderID', there are no corresponding entries for 33.769352% of entries in 'SalesOrderID' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and STORE__STAGING_TABLE_5 over 'SalesPersonID' and 'SalesPersonID', there are no corresponding entries for 84.941548% of entries in 'SalesPersonID' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.


In [27]:
pipe2.fit(container.train)

Checking data model...


Staging...
[========================================] 100%

INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and SALES_ORDER_REASON__STAGING_TABLE_4 over 'SalesOrderID' and 'SalesOrderID', there are no corresponding entries for 33.769352% of entries in 'SalesOrderID' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and STORE__STAGING_TABLE_5 over 'SalesPersonID' and 'SalesPersonID', there are no corresponding entries for 84.941548% of entries in 'SalesPersonID' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

Relboost: Training features...
[========================================] 100%

Relboost: Building features...
[========================================] 100%

XGBoost: Trai

Pipeline(data_model='population',
         feature_learners=['Relboost'],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=['product', 'sales_order_detail', 'sales_order_header',
                     'sales_order_reason', 'special_offer', 'store'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Seasonal', 'Mapping'],
         share_selected_features=0.5,
         tags=['relboost', 'container-EORt4D'])

url: http://localhost:1709/#/getpipeline/adventure_works/tOLvI0/0/

### 2.4 Model evaluation

In [28]:
pipe1.score(container.test)



Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

FastProp: Building features...
[========================================] 100%



,date time,set used,target,accuracy,auc,cross entropy
0,2021-08-20 11:13:18,train,churn,0.9123,0.9708,0.2244
1,2021-08-20 11:14:50,test,churn,0.9062,0.9658,0.2375


In [29]:
pipe2.score(container.test)



Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

Relboost: Building features...
[========================================] 100%



,date time,set used,target,accuracy,auc,cross entropy
0,2021-08-20 11:14:47,train,churn,0.9333,0.9839,0.1644
1,2021-08-20 11:14:54,test,churn,0.9296,0.9798,0.1832


### 2.5 Productionization in Apache Spark

It is possible to productionize the pipeline by transpiling the features into production-ready SQL code.

In [30]:
pipe1.features.to_sql(dialect=getml.pipeline.dialect.spark_sql).save("adventure_works1_spark_sql")

In [31]:
pipe2.features.to_sql(dialect=getml.pipeline.dialect.spark_sql).save("adventure_works2_spark_sql")

In [32]:
spark = SparkSession(sc)

In [33]:
population_spark = container.train.population.to_pyspark(spark, name="population")

In [34]:
product_spark = product.to_pyspark(spark, name="product")
sales_order_detail_spark = sales_order_detail.to_pyspark(spark, name="sales_order_detail")
sales_order_header_spark = sales_order_header.to_pyspark(spark, name="sales_order_header")
sales_order_reason_spark = sales_order_reason.to_pyspark(spark, name="sales_order_reason")
special_offer_spark = special_offer.to_pyspark(spark, name="special_offer")
store = store.to_pyspark(spark, name="store")

In [40]:
begin = time.time()
getml.spark.execute(spark, "adventure_works1_spark_sql")
end = time.time()

spark_runtime1 = datetime.timedelta(seconds=end - begin)

2021-08-20 12:05:21,172 Executing adventure_works1_spark_sql/0001_population__staging_table_1.sql...
2021-08-20 12:05:21,741 Executing adventure_works1_spark_sql/0002_sales_order_detail__staging_table_2.sql...
2021-08-20 12:05:25,786 Executing adventure_works1_spark_sql/0003_sales_order_header__staging_table_3.sql...
2021-08-20 12:05:26,519 Executing adventure_works1_spark_sql/0004_sales_order_reason__staging_table_4.sql...
2021-08-20 12:05:26,854 Executing adventure_works1_spark_sql/0005_store__staging_table_5.sql...
2021-08-20 12:05:27,384 Executing adventure_works1_spark_sql/0006_onlineorderflag__mapping_1_target_1_avg.sql...
2021-08-20 12:05:28,573 Executing adventure_works1_spark_sql/0007_salespersonidcat__mapping_1_target_1_avg.sql...
2021-08-20 12:05:29,906 Executing adventure_works1_spark_sql/0008_territoryidcat__mapping_1_target_1_avg.sql...
2021-08-20 12:05:31,235 Executing adventure_works1_spark_sql/0009_shipmethodid__mapping_1_target_1_avg.sql...
2021-08-20 12:05:32,656 Exe

2021-08-20 12:09:40,484 Executing adventure_works1_spark_sql/0090_feature_1_66.sql...
2021-08-20 12:09:43,441 Executing adventure_works1_spark_sql/0091_feature_1_67.sql...
2021-08-20 12:09:46,813 Executing adventure_works1_spark_sql/0092_feature_1_68.sql...
2021-08-20 12:09:49,765 Executing adventure_works1_spark_sql/0093_feature_1_69.sql...
2021-08-20 12:09:54,081 Executing adventure_works1_spark_sql/0094_feature_1_70.sql...
2021-08-20 12:09:57,043 Executing adventure_works1_spark_sql/0095_feature_1_71.sql...
2021-08-20 12:10:00,061 Executing adventure_works1_spark_sql/0096_feature_1_72.sql...
2021-08-20 12:10:03,492 Executing adventure_works1_spark_sql/0097_feature_1_73.sql...
2021-08-20 12:10:06,441 Executing adventure_works1_spark_sql/0098_feature_1_74.sql...
2021-08-20 12:10:12,766 Executing adventure_works1_spark_sql/0099_feature_1_75.sql...
2021-08-20 12:10:16,948 Executing adventure_works1_spark_sql/0100_feature_1_76.sql...
2021-08-20 12:10:22,567 Executing adventure_works1_spa

2021-08-20 12:16:41,957 Executing adventure_works1_spark_sql/0185_feature_1_161.sql...
2021-08-20 12:16:46,978 Executing adventure_works1_spark_sql/0186_feature_1_162.sql...
2021-08-20 12:16:52,211 Executing adventure_works1_spark_sql/0187_feature_1_163.sql...
2021-08-20 12:16:57,096 Executing adventure_works1_spark_sql/0188_feature_1_164.sql...
2021-08-20 12:17:02,205 Executing adventure_works1_spark_sql/0189_feature_1_165.sql...
2021-08-20 12:17:07,010 Executing adventure_works1_spark_sql/0190_feature_1_166.sql...
2021-08-20 12:17:12,121 Executing adventure_works1_spark_sql/0191_feature_1_167.sql...
2021-08-20 12:17:17,357 Executing adventure_works1_spark_sql/0192_feature_1_168.sql...
2021-08-20 12:17:20,705 Executing adventure_works1_spark_sql/0193_feature_1_169.sql...
2021-08-20 12:17:23,867 Executing adventure_works1_spark_sql/0194_feature_1_170.sql...
2021-08-20 12:17:27,222 Executing adventure_works1_spark_sql/0195_feature_1_171.sql...
2021-08-20 12:17:32,369 Executing adventure

2021-08-20 12:23:40,536 Executing adventure_works1_spark_sql/0280_feature_1_256.sql...
2021-08-20 12:23:43,769 Executing adventure_works1_spark_sql/0281_feature_1_257.sql...
2021-08-20 12:23:49,353 Executing adventure_works1_spark_sql/0282_feature_1_258.sql...
2021-08-20 12:23:52,824 Executing adventure_works1_spark_sql/0283_feature_1_259.sql...
2021-08-20 12:23:57,570 Executing adventure_works1_spark_sql/0284_feature_1_260.sql...
2021-08-20 12:24:00,938 Executing adventure_works1_spark_sql/0285_feature_1_261.sql...
2021-08-20 12:24:05,811 Executing adventure_works1_spark_sql/0286_feature_1_262.sql...
2021-08-20 12:24:11,055 Executing adventure_works1_spark_sql/0287_feature_1_263.sql...
2021-08-20 12:24:15,892 Executing adventure_works1_spark_sql/0288_feature_1_264.sql...
2021-08-20 12:24:20,853 Executing adventure_works1_spark_sql/0289_feature_1_265.sql...
2021-08-20 12:24:25,706 Executing adventure_works1_spark_sql/0290_feature_1_266.sql...
2021-08-20 12:24:30,610 Executing adventure

2021-08-20 12:30:52,662 Executing adventure_works1_spark_sql/0375_feature_1_351.sql...
2021-08-20 12:30:57,662 Executing adventure_works1_spark_sql/0376_feature_1_352.sql...
2021-08-20 12:31:02,931 Executing adventure_works1_spark_sql/0377_feature_1_353.sql...
2021-08-20 12:31:08,025 Executing adventure_works1_spark_sql/0378_feature_1_354.sql...
2021-08-20 12:31:12,993 Executing adventure_works1_spark_sql/0379_feature_1_355.sql...
2021-08-20 12:31:18,145 Executing adventure_works1_spark_sql/0380_feature_1_356.sql...
2021-08-20 12:31:23,097 Executing adventure_works1_spark_sql/0381_feature_1_357.sql...
2021-08-20 12:31:28,720 Executing adventure_works1_spark_sql/0382_feature_1_358.sql...
2021-08-20 12:31:33,633 Executing adventure_works1_spark_sql/0383_feature_1_359.sql...
2021-08-20 12:31:36,858 Executing adventure_works1_spark_sql/0384_feature_1_360.sql...
2021-08-20 12:31:40,002 Executing adventure_works1_spark_sql/0385_feature_1_361.sql...
2021-08-20 12:31:45,150 Executing adventure

In [42]:
begin = time.time()
getml.spark.execute(spark, "adventure_works2_spark_sql")
end = time.time()

spark_runtime2 = datetime.timedelta(seconds=end - begin)

2021-08-20 12:39:13,845 Executing adventure_works2_spark_sql/0001_population__staging_table_1.sql...
2021-08-20 12:39:14,702 Executing adventure_works2_spark_sql/0002_sales_order_detail__staging_table_2.sql...
2021-08-20 12:39:19,739 Executing adventure_works2_spark_sql/0003_sales_order_header__staging_table_3.sql...
2021-08-20 12:39:20,512 Executing adventure_works2_spark_sql/0004_sales_order_reason__staging_table_4.sql...
2021-08-20 12:39:21,279 Executing adventure_works2_spark_sql/0005_store__staging_table_5.sql...
2021-08-20 12:39:21,680 Executing adventure_works2_spark_sql/0006_onlineorderflag__mapping_1_target_1_avg.sql...
2021-08-20 12:39:23,549 Executing adventure_works2_spark_sql/0007_salespersonidcat__mapping_1_target_1_avg.sql...
2021-08-20 12:39:25,715 Executing adventure_works2_spark_sql/0008_territoryidcat__mapping_1_target_1_avg.sql...
2021-08-20 12:39:27,385 Executing adventure_works2_spark_sql/0009_shipmethodid__mapping_1_target_1_avg.sql...
2021-08-20 12:39:31,252 Exe

2021-08-20 12:42:49,971 Executing adventure_works2_spark_sql/0074_feature_1_18.sql...
2021-08-20 12:42:54,552 Executing adventure_works2_spark_sql/0075_feature_1_19.sql...
2021-08-20 12:42:58,269 Executing adventure_works2_spark_sql/0076_feature_1_20.sql...
2021-08-20 12:43:02,146 Executing adventure_works2_spark_sql/0077_feature_1_21.sql...
2021-08-20 12:43:06,001 Executing adventure_works2_spark_sql/0078_feature_1_22.sql...
2021-08-20 12:43:09,815 Executing adventure_works2_spark_sql/0079_feature_1_23.sql...
2021-08-20 12:43:13,937 Executing adventure_works2_spark_sql/0080_feature_1_24.sql...
2021-08-20 12:43:18,232 Executing adventure_works2_spark_sql/0081_feature_1_25.sql...
2021-08-20 12:43:22,358 Executing adventure_works2_spark_sql/0082_feature_1_26.sql...
2021-08-20 12:43:26,240 Executing adventure_works2_spark_sql/0083_feature_1_27.sql...
2021-08-20 12:43:30,519 Executing adventure_works2_spark_sql/0084_feature_1_28.sql...
2021-08-20 12:43:35,035 Executing adventure_works2_spa

In [43]:
begin = time.time()
features1 = pipe1.transform(container.train)
end = time.time()

getml_runtime1 = datetime.timedelta(seconds=end - begin)



Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

FastProp: Building features...
[========================================] 100%



In [44]:
begin = time.time()
features1 = pipe2.transform(container.train)
end = time.time()

getml_runtime2 = datetime.timedelta(seconds=end - begin)



Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

Relboost: Building features...
[========================================] 100%



In [45]:
spark_runtime1/getml_runtime1

218.39108985638953

In [46]:
spark_runtime2/getml_runtime2

68.7643293703773

## 3. Conclusion

We demonstrated how getML can be used for a classic customer churn project and shown how it can be productionized in Apache Spark.

## References

Motl, Jan, and Oliver Schulte. "The CTU prague relational learning repository." arXiv preprint arXiv:1511.03086 (2015).

# Next Steps

This tutorial benchmarked getML against academic state-of-the-art algorithms from relational learning literature and getML's qualities with respect to categorical data.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.